# CIGRE MV Powerflow with Profiles Test

## Run simulation

In [ ]:
import requests
import glob

def download_grid_data(name, url):
    with open(name, 'wb') as out_file:
        content = requests.get(url, stream=True).content
        out_file.write(content)

url = 'https://raw.githubusercontent.com/dpsim-simulator/cim-grid-data/master/CIGRE_MV/NEPLAN/CIGRE_MV_no_tapchanger_With_LoadFlow_Results/Rootnet_FULL_NE_06J16h'
filename = 'CIGRE-MV'
download_grid_data(filename+'_EQ.xml', url+'_EQ.xml')
download_grid_data(filename+'_TP.xml', url+'_TP.xml')
download_grid_data(filename+'_SV.xml', url+'_SV.xml')

files = glob.glob(filename+'_*.xml')
print(files)

In [ ]:
import os

profiles = ['Load_H_1','Load_H_3','Load_H_4','Load_H_5','Load_H_6','Load_H_8','Load_H_10','Load_H_11','Load_H_12','Load_H_14','Load_I_1','Load_I_3','Load_I_7','Load_I_9','Load_I_10','Load_I_12','Load_I_13','Load_I_14']
for profile in profiles:
    filename = './profiles/'+profile+'.csv'
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    download_grid_data(filename, 'https://raw.githubusercontent.com/dpsim-simulator/example-profile-data/master/CIGRE_MV_NoTap/load_profiles/'+profile+'.csv')
    
profile_files = glob.glob('profiles/Load*.csv')


In [ ]:
import pathlib

profile_path = str(pathlib.Path().resolve())+'/profiles/'
print(profile_path)

In [ ]:
import dpsimpy

In [ ]:
name = 'CIGRE-MV-Profiles'
reader = dpsimpy.CIMReader(name)
system = reader.loadCIM(50, files, dpsimpy.Domain.SP, dpsimpy.PhaseType.Single, dpsimpy.GeneratorType.PVNode)

In [ ]:
assignList = { }
assignList['LOAD-H-1'] = 'Load_H_1'
assignList['LOAD-H-3'] = 'Load_H_3'
assignList['LOAD-H-4'] = 'Load_H_4'
assignList['LOAD-H-5'] = 'Load_H_5'
assignList['LOAD-H-6'] = 'Load_H_6'
assignList['LOAD-H-8'] = 'Load_H_8'
assignList['LOAD-H-10'] = 'Load_H_10'
assignList['LOAD-H-11'] = 'Load_H_11'
assignList['LOAD-H-12'] = 'Load_H_12'
assignList['LOAD-H-14'] = 'Load_H_14'
assignList['LOAD-I-1'] = 'Load_I_1'
assignList['LOAD-I-3'] = 'Load_I_3'
assignList['LOAD-I-7'] = 'Load_I_7'
assignList['LOAD-I-9'] = 'Load_I_9'
assignList['LOAD-I-10'] = 'Load_I_10'
assignList['LOAD-I-12'] = 'Load_I_12'
assignList['LOAD-I-13'] = 'Load_I_13'
assignList['LOAD-I-14'] = 'Load_I_14'

In [ ]:
csvreader = dpsimpy.CSVReader(name, profile_path, assignList, dpsimpy.LogLevel.info)
csvreader.assignLoadProfile(system, 0, 1, 300, dpsimpy.CSVReaderMode.MANUAL, dpsimpy.CSVReaderFormat.SECONDS)

In [ ]:
system

In [ ]:
sim = dpsimpy.Simulation(name)
sim.set_system(system)
sim.set_domain(dpsimpy.Domain.SP)
sim.set_solver(dpsimpy.Solver.NRP)
sim.set_time_step(1)
sim.set_final_time(300)

logger = dpsimpy.Logger(name)
sim.add_logger(logger)

intf = dpsimpy.InterfaceShmem()
sim.add_interface(intf)

In [ ]:
obj_list = system.list_idobjects()
node_list = {k: v for k, v in obj_list.items() if v == 'SimNode<std::complex<double> >'}
node_list

In [ ]:
for i in range(14):
    sim.export_attribute('N'+str(i), 'v', i, dpsimpy.AttrModifier.mag)
    sim.export_attribute('N'+str(i), 'v', i+1, dpsimpy.AttrModifier.phase)

In [ ]:
for node in system.nodes:
    logger.log_attribute(node.name()+'.V', 'v', node)

In [ ]:
sim.run()